In [1]:
# finding max generation kwh per $

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Define all available roof spaces with their characteristics
AVAILABLE_ROOFS = [
    {
        'id': 'shed',
        'name': 'Shed',
        'max_capacity_kw': 14.5,
        'tilt': 10,
        'azimuth': 23.82,
        'shading': 7.839,
        'array_type': 1,  # Fixed roof mount
        'cost_factor': 1.0  # Base cost (no premium)
    },
    {
        'id': 'house_left',
        'name': 'House on left',
        'max_capacity_kw': 15.0,
        'tilt': 10.0,
        'azimuth': 128.17,
        'shading': 0,
        'array_type': 1,
        'cost_factor': 1.0
    },
    {
        'id': 'house_right',
        'name': 'House on right',
        'max_capacity_kw': 30.0,
        'tilt': 7.0,
        'azimuth': 20.01,
        'shading': 14.03,
        'array_type': 1,
        'cost_factor': 1.0
    },
    {
        'id': 'residential_accom',
        'name': 'Residential Accommodation',
        'max_capacity_kw': 33.0,
        'tilt': 20.0,
        'azimuth': 40.42,
        'shading': 0,
        'array_type': 1,
        'cost_factor': 1.0
    },
    {
        'id': 'small_shed',
        'name': 'Small shed',
        'max_capacity_kw': 9.5,
        'tilt': 20.0,
        'azimuth': 20.55,
        'shading': 6.4474,
        'array_type': 1,
        'cost_factor': 1.0
    },
    {
        'id': 'ground_mounted',
        'name': 'Ground-Mounted Optimal Angle Array',
        'max_capacity_kw': float('inf'),  # Unlimited capacity
        'tilt': 30.0,
        'azimuth': 5.0,
        'shading': 0.0,
        'array_type': 0,  # Fixed open rack (ground mounted)
        'cost_factor': 1.15  # 15% premium
    }
]

# Function to calculate PV cost per kW based on system size
def calculate_pv_cost_per_kw(system_size_kw):
    """Calculate the cost per kW based on system size"""
    cost = 1047.3 * np.exp(-0.002 * system_size_kw)
    # Set minimum cost to 750
    return max(cost, 750)

# Simplified PV generation model based on roof parameters
def estimate_generation(roof, baseline_generation=1500):
    """
    Estimate annual generation for a roof configuration
    
    Parameters:
    - roof: Dictionary with roof parameters
    - baseline_generation: kWh per kW at ideal conditions
    
    Returns:
    - Estimated kWh per kW per year
    """
    # Azimuth factor (North = 0, East = 90, South = 180, West = 270)
    # Optimal is South (180 in Northern hemisphere, or 0/360 in Southern hemisphere)
    # Assuming we're in the Southern hemisphere, best is North (0 degrees)
    azimuth_optimal = 0  # North in Southern hemisphere
    azimuth_factor = 1 - (abs(roof['azimuth'] - azimuth_optimal) / 180) * 0.15
    
    # Tilt factor (optimal tilt depends on latitude)
    # Without knowing the exact latitude, we'll assume optimal is around 30 degrees
    tilt_optimal = 30
    tilt_factor = 1 - (abs(roof['tilt'] - tilt_optimal) / 30) * 0.1
    
    # Shading factor
    shading_factor = 1 - (roof['shading'] / 100)
    
    # Array type factor
    array_type_factor = 1.05 if roof['array_type'] == 0 else 1.0  # Ground mounted has better cooling
    
    # Calculate the adjusted generation
    adjusted_generation = baseline_generation * azimuth_factor * tilt_factor * shading_factor * array_type_factor
    
    return adjusted_generation

# Calculate economic metrics for each roof option
def analyze_roof_economics(roofs, baseline_generation=1500):
    """Analyze the economic performance of each available roof"""
    roof_metrics = []
    
    for roof in roofs:
        # Estimate generation for this roof (kWh per kW per year)
        generation_per_kw = estimate_generation(roof, baseline_generation)
        
        # Calculate cost per kW for 1 kW system (to compare unit economics)
        unit_cost_per_kw = calculate_pv_cost_per_kw(1.0) * roof['cost_factor']
        
        # For max capacity system (to see economies of scale)
        max_system_size = min(roof['max_capacity_kw'], 100) if roof['max_capacity_kw'] != float('inf') else 100
        max_cost_per_kw = calculate_pv_cost_per_kw(max_system_size) * roof['cost_factor']
        
        # Calculate economic metrics
        unit_generation_per_dollar = generation_per_kw / unit_cost_per_kw
        max_generation_per_dollar = generation_per_kw / max_cost_per_kw
        
        # Store metrics
        roof_metrics.append({
            'roof': roof,
            'generation_per_kw': generation_per_kw,
            'unit_cost_per_kw': unit_cost_per_kw,
            'max_cost_per_kw': max_cost_per_kw,
            'unit_generation_per_dollar': unit_generation_per_dollar,
            'max_generation_per_dollar': max_generation_per_dollar
        })
    
    return roof_metrics

# Function to allocate PV capacity to roofs in order of economic value
def allocate_pv_to_roofs(roofs_metrics, total_capacity_kw=100):
    """Allocate PV capacity to roofs in order of economic value"""
    # Sort roofs by economic value (generation per dollar)
    sorted_roofs = sorted(roofs_metrics, key=lambda x: x['max_generation_per_dollar'], reverse=True)
    
    # Allocate capacity
    allocated_capacity = []
    remaining_capacity = total_capacity_kw
    
    for roof_data in sorted_roofs:
        roof = roof_data['roof']
        if remaining_capacity <= 0:
            break
            
        # Allocate capacity to this roof (up to its maximum)
        max_allocation = roof['max_capacity_kw'] if roof['max_capacity_kw'] != float('inf') else remaining_capacity
        allocation = min(remaining_capacity, max_allocation)
        
        if allocation > 0:
            # Calculate actual cost with economies of scale
            actual_cost_per_kw = calculate_pv_cost_per_kw(allocation) * roof['cost_factor']
            total_cost = actual_cost_per_kw * allocation
            
            # Calculate estimated generation
            annual_generation = allocation * roof_data['generation_per_kw']
            
            allocated_capacity.append({
                'roof': roof,
                'allocated_kw': allocation,
                'cost_per_kw': actual_cost_per_kw,
                'total_cost': total_cost,
                'annual_generation_kwh': annual_generation,
                'generation_per_dollar': annual_generation / total_cost
            })
            
            # Reduce remaining capacity
            remaining_capacity -= allocation
    
    return allocated_capacity

# Function to create a clear visual of the allocation
def visualize_allocation(allocation):
    """Create a bar chart of the allocation"""
    names = [item['roof']['name'] for item in allocation]
    capacities = [item['allocated_kw'] for item in allocation]
    
    plt.figure(figsize=(12, 6))
    bars = plt.bar(names, capacities)
    
    # Add value labels on top of bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                 f'{height:.1f} kW',
                 ha='center', va='bottom')
    
    plt.xlabel('Roof Location')
    plt.ylabel('Allocated Capacity (kW)')
    plt.title('PV Capacity Allocation by Roof Location')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Save the figure
    plt.savefig('pv_allocation.png')
    plt.close()

# Main execution
def main():
    # Analyze roof economics
    roof_economics = analyze_roof_economics(AVAILABLE_ROOFS)
    
    # Print the results in descending order of economic value
    print("Roof Economic Ranking (best to worst, based on generation per dollar):")
    for i, roof_data in enumerate(sorted(roof_economics, key=lambda x: x['max_generation_per_dollar'], reverse=True)):
        roof = roof_data['roof']
        print(f"{i+1}. {roof['name']}:")
        print(f"   - Generation: {roof_data['generation_per_kw']:.2f} kWh/kW/year")
        print(f"   - Unit Cost: ${roof_data['unit_cost_per_kw']:.2f}/kW") 
        print(f"   - Max Capacity Cost: ${roof_data['max_cost_per_kw']:.2f}/kW")
        print(f"   - Economic Value: {roof_data['max_generation_per_dollar']:.4f} kWh/$")
        print(f"   - Max Capacity: {roof['max_capacity_kw'] if roof['max_capacity_kw'] != float('inf') else 'Unlimited'} kW")
        print()
    
    # Allocate capacity and print the allocation
    print("\n=== Optimal PV Allocation for 100kW System ===")
    allocation = allocate_pv_to_roofs(roof_economics, 100)
    total_allocated = sum(item['allocated_kw'] for item in allocation)
    total_cost = sum(item['total_cost'] for item in allocation)
    total_generation = sum(item['annual_generation_kwh'] for item in allocation)
    
    print(f"Total Allocated Capacity: {total_allocated:.2f} kW")
    print(f"Total System Cost: ${total_cost:.2f}")
    print(f"Total Annual Generation: {total_generation:.2f} kWh")
    print(f"Overall System Efficiency: {total_generation / total_allocated:.2f} kWh/kW/year")
    print(f"Overall System Economic Value: {total_generation / total_cost:.4f} kWh/$")
    print()
    
    print("Allocation Details:")
    for item in allocation:
        roof = item['roof']
        print(f"- {roof['name']}: {item['allocated_kw']:.2f} kW (${item['total_cost']:.2f}, {item['annual_generation_kwh']:.2f} kWh/year)")
    
    # Try to create a visualization (comment this out if matplotlib is not available)
    try:
        visualize_allocation(allocation)
        print("\nVisualization saved as 'pv_allocation.png'")
    except Exception as e:
        print(f"\nCould not create visualization: {e}")

if __name__ == "__main__":
    main()

Roof Economic Ranking (best to worst, based on generation per dollar):
1. Ground-Mounted Optimal Angle Array:
   - Generation: 1568.44 kWh/kW/year
   - Unit Cost: $1201.99/kW
   - Max Capacity Cost: $986.08/kW
   - Economic Value: 1.5906 kWh/$
   - Max Capacity: Unlimited kW

2. Residential Accommodation:
   - Generation: 1401.16 kWh/kW/year
   - Unit Cost: $1045.21/kW
   - Max Capacity Cost: $980.41/kW
   - Economic Value: 1.4292 kWh/$
   - Max Capacity: 33.0 kW

3. Small shed:
   - Generation: 1333.28 kWh/kW/year
   - Unit Cost: $1045.21/kW
   - Max Capacity Cost: $1027.59/kW
   - Economic Value: 1.2975 kWh/$
   - Max Capacity: 9.5 kW

4. Shed:
   - Generation: 1264.64 kWh/kW/year
   - Unit Cost: $1045.21/kW
   - Max Capacity Cost: $1017.36/kW
   - Economic Value: 1.2431 kWh/$
   - Max Capacity: 14.5 kW

5. House on left:
   - Generation: 1250.47 kWh/kW/year
   - Unit Cost: $1045.21/kW
   - Max Capacity Cost: $1016.35/kW
   - Economic Value: 1.2304 kWh/$
   - Max Capacity: 15.0 kW

6

In [4]:
# premium for mounted is 25%

import numpy as np
import matplotlib.pyplot as plt

# Define all available roof spaces with their characteristics
AVAILABLE_ROOFS = [
    {
        'id': 'shed',
        'name': 'Shed',
        'max_capacity_kw': 14.5,
        'tilt': 10,
        'azimuth': 23.82,
        'shading': 7.839,
        'array_type': 1,  # Fixed roof mount
        'cost_factor': 1.0  # Base cost (no premium)
    },
    {
        'id': 'house_left',
        'name': 'House on left',
        'max_capacity_kw': 15.0,
        'tilt': 10.0,
        'azimuth': 128.17,
        'shading': 0,
        'array_type': 1,
        'cost_factor': 1.0
    },
    {
        'id': 'house_right',
        'name': 'House on right',
        'max_capacity_kw': 30.0,
        'tilt': 7.0,
        'azimuth': 20.01,
        'shading': 14.03,
        'array_type': 1,
        'cost_factor': 1.0
    },
    {
        'id': 'residential_accom',
        'name': 'Residential Accommodation',
        'max_capacity_kw': 33.0,
        'tilt': 20.0,
        'azimuth': 40.42,
        'shading': 0,
        'array_type': 1,
        'cost_factor': 1.0
    },
    {
        'id': 'small_shed',
        'name': 'Small shed',
        'max_capacity_kw': 9.5,
        'tilt': 20.0,
        'azimuth': 20.55,
        'shading': 6.4474,
        'array_type': 1,
        'cost_factor': 1.0
    },
    {
        'id': 'ground_mounted',
        'name': 'Ground-Mounted Optimal Angle Array',
        'max_capacity_kw': float('inf'),  # Unlimited capacity
        'tilt': 30.0,
        'azimuth': 5.0,
        'shading': 0.0,
        'array_type': 0,  # Fixed open rack (ground mounted)
        'cost_factor': 1.30  # 15% premium
    }
]

# Function to calculate PV cost per kW based on system size
def calculate_pv_cost_per_kw(system_size_kw):
    """Calculate the cost per kW based on system size"""
    cost = 1047.3 * np.exp(-0.002 * system_size_kw)
    # Set minimum cost to 750
    return max(cost, 750)

# Simplified PV generation model based on roof parameters
def estimate_generation(roof, baseline_generation=1500):
    """
    Estimate annual generation for a roof configuration
    
    Parameters:
    - roof: Dictionary with roof parameters
    - baseline_generation: kWh per kW at ideal conditions
    
    Returns:
    - Estimated kWh per kW per year
    """
    # Azimuth factor (North = 0, East = 90, South = 180, West = 270)
    # Optimal is South (180 in Northern hemisphere, or 0/360 in Southern hemisphere)
    # Assuming we're in the Southern hemisphere, best is North (0 degrees)
    azimuth_optimal = 0  # North in Southern hemisphere
    azimuth_factor = 1 - (abs(roof['azimuth'] - azimuth_optimal) / 180) * 0.15
    
    # Tilt factor (optimal tilt depends on latitude)
    # Without knowing the exact latitude, we'll assume optimal is around 30 degrees
    tilt_optimal = 30
    tilt_factor = 1 - (abs(roof['tilt'] - tilt_optimal) / 30) * 0.1
    
    # Shading factor
    shading_factor = 1 - (roof['shading'] / 100)
    
    # Array type factor
    array_type_factor = 1.05 if roof['array_type'] == 0 else 1.0  # Ground mounted has better cooling
    
    # Calculate the adjusted generation
    adjusted_generation = baseline_generation * azimuth_factor * tilt_factor * shading_factor * array_type_factor
    
    return adjusted_generation

# Calculate economic metrics for each roof option
def analyze_roof_economics(roofs, baseline_generation=1500):
    """Analyze the economic performance of each available roof"""
    roof_metrics = []
    
    for roof in roofs:
        # Estimate generation for this roof (kWh per kW per year)
        generation_per_kw = estimate_generation(roof, baseline_generation)
        
        # Calculate cost per kW for 1 kW system (to compare unit economics)
        unit_cost_per_kw = calculate_pv_cost_per_kw(1.0) * roof['cost_factor']
        
        # For max capacity system (to see economies of scale)
        max_system_size = min(roof['max_capacity_kw'], 100) if roof['max_capacity_kw'] != float('inf') else 100
        max_cost_per_kw = calculate_pv_cost_per_kw(max_system_size) * roof['cost_factor']
        
        # Calculate economic metrics
        unit_generation_per_dollar = generation_per_kw / unit_cost_per_kw
        max_generation_per_dollar = generation_per_kw / max_cost_per_kw
        
        # Store metrics
        roof_metrics.append({
            'roof': roof,
            'generation_per_kw': generation_per_kw,
            'unit_cost_per_kw': unit_cost_per_kw,
            'max_cost_per_kw': max_cost_per_kw,
            'unit_generation_per_dollar': unit_generation_per_dollar,
            'max_generation_per_dollar': max_generation_per_dollar
        })
    
    return roof_metrics

# Function to allocate PV capacity to roofs in order of economic value
def allocate_pv_to_roofs(roofs_metrics, total_capacity_kw=100):
    """Allocate PV capacity to roofs in order of economic value"""
    # Sort roofs by economic value (generation per dollar)
    sorted_roofs = sorted(roofs_metrics, key=lambda x: x['max_generation_per_dollar'], reverse=True)
    
    # Allocate capacity
    allocated_capacity = []
    remaining_capacity = total_capacity_kw
    
    for roof_data in sorted_roofs:
        roof = roof_data['roof']
        if remaining_capacity <= 0:
            break
            
        # Allocate capacity to this roof (up to its maximum)
        max_allocation = roof['max_capacity_kw'] if roof['max_capacity_kw'] != float('inf') else remaining_capacity
        allocation = min(remaining_capacity, max_allocation)
        
        if allocation > 0:
            # Calculate actual cost with economies of scale
            actual_cost_per_kw = calculate_pv_cost_per_kw(allocation) * roof['cost_factor']
            total_cost = actual_cost_per_kw * allocation
            
            # Calculate estimated generation
            annual_generation = allocation * roof_data['generation_per_kw']
            
            allocated_capacity.append({
                'roof': roof,
                'allocated_kw': allocation,
                'cost_per_kw': actual_cost_per_kw,
                'total_cost': total_cost,
                'annual_generation_kwh': annual_generation,
                'generation_per_dollar': annual_generation / total_cost
            })
            
            # Reduce remaining capacity
            remaining_capacity -= allocation
    
    return allocated_capacity

# Function to create a clear visual of the allocation
def visualize_allocation(allocation):
    """Create a bar chart of the allocation"""
    names = [item['roof']['name'] for item in allocation]
    capacities = [item['allocated_kw'] for item in allocation]
    
    plt.figure(figsize=(12, 6))
    bars = plt.bar(names, capacities)
    
    # Add value labels on top of bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                 f'{height:.1f} kW',
                 ha='center', va='bottom')
    
    plt.xlabel('Roof Location')
    plt.ylabel('Allocated Capacity (kW)')
    plt.title('PV Capacity Allocation by Roof Location')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Save the figure
    plt.savefig('pv_allocation.png')
    plt.close()

# Main execution
def main():
    # Analyze roof economics
    roof_economics = analyze_roof_economics(AVAILABLE_ROOFS)
    
    # Print the results in descending order of economic value
    print("Roof Economic Ranking (best to worst, based on generation per dollar):")
    for i, roof_data in enumerate(sorted(roof_economics, key=lambda x: x['max_generation_per_dollar'], reverse=True)):
        roof = roof_data['roof']
        print(f"{i+1}. {roof['name']}:")
        print(f"   - Generation: {roof_data['generation_per_kw']:.2f} kWh/kW/year")
        print(f"   - Unit Cost: ${roof_data['unit_cost_per_kw']:.2f}/kW") 
        print(f"   - Max Capacity Cost: ${roof_data['max_cost_per_kw']:.2f}/kW")
        print(f"   - Economic Value: {roof_data['max_generation_per_dollar']:.4f} kWh/$")
        print(f"   - Max Capacity: {roof['max_capacity_kw'] if roof['max_capacity_kw'] != float('inf') else 'Unlimited'} kW")
        print()
    
    # Allocate capacity and print the allocation
    print("\n=== Optimal PV Allocation for 100kW System ===")
    allocation = allocate_pv_to_roofs(roof_economics, 100)
    total_allocated = sum(item['allocated_kw'] for item in allocation)
    total_cost = sum(item['total_cost'] for item in allocation)
    total_generation = sum(item['annual_generation_kwh'] for item in allocation)
    
    print(f"Total Allocated Capacity: {total_allocated:.2f} kW")
    print(f"Total System Cost: ${total_cost:.2f}")
    print(f"Total Annual Generation: {total_generation:.2f} kWh")
    print(f"Overall System Efficiency: {total_generation / total_allocated:.2f} kWh/kW/year")
    print(f"Overall System Economic Value: {total_generation / total_cost:.4f} kWh/$")
    print()
    
    print("Allocation Details:")
    for item in allocation:
        roof = item['roof']
        print(f"- {roof['name']}: {item['allocated_kw']:.2f} kW (${item['total_cost']:.2f}, {item['annual_generation_kwh']:.2f} kWh/year)")
    
    # Try to create a visualization (comment this out if matplotlib is not available)
    try:
        visualize_allocation(allocation)
        print("\nVisualization saved as 'pv_allocation.png'")
    except Exception as e:
        print(f"\nCould not create visualization: {e}")

if __name__ == "__main__":
    main()

Roof Economic Ranking (best to worst, based on generation per dollar):
1. Residential Accommodation:
   - Generation: 1401.16 kWh/kW/year
   - Unit Cost: $1045.21/kW
   - Max Capacity Cost: $980.41/kW
   - Economic Value: 1.4292 kWh/$
   - Max Capacity: 33.0 kW

2. Ground-Mounted Optimal Angle Array:
   - Generation: 1568.44 kWh/kW/year
   - Unit Cost: $1358.77/kW
   - Max Capacity Cost: $1114.69/kW
   - Economic Value: 1.4071 kWh/$
   - Max Capacity: Unlimited kW

3. Small shed:
   - Generation: 1333.28 kWh/kW/year
   - Unit Cost: $1045.21/kW
   - Max Capacity Cost: $1027.59/kW
   - Economic Value: 1.2975 kWh/$
   - Max Capacity: 9.5 kW

4. Shed:
   - Generation: 1264.64 kWh/kW/year
   - Unit Cost: $1045.21/kW
   - Max Capacity Cost: $1017.36/kW
   - Economic Value: 1.2431 kWh/$
   - Max Capacity: 14.5 kW

5. House on left:
   - Generation: 1250.47 kWh/kW/year
   - Unit Cost: $1045.21/kW
   - Max Capacity Cost: $1016.35/kW
   - Economic Value: 1.2304 kWh/$
   - Max Capacity: 15.0 kW

